In [6]:

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer
from joblib import load

from sqlalchemy import create_engine
import pymssql
from datetime import datetime

from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass

KEY = "PKX210XR6N17LF7WIWC8"
SECRET_KEY = "Cc6VXW58xRrsMQ98kbkZvw5dAOKKPMZaiRpLGraG"
numstocks = 200

qt = QuantileTransformer(output_distribution="normal")
pipe = load("files/linear_model_2023-01-20.joblib")

c:\Users\keb7\repos\mgmt767-binder\venv\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator PolynomialFeatures from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\keb7\repos\mgmt767-binder\venv\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\keb7\repos\mgmt767-binder\venv\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator Pipeline from version 1.1.2 when using version 1.2.0. This might lead to breaking

## Rank stocks

In [ ]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_style("whitegrid")

## Build Feature Dataset

### SQL 

- Don't need much history.  Start here in 2022.

In [ ]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
driver = 'SQL+Server'
string = f"mssql+pyodbc://{username}:{password}@{server}/{database}" 
try: 
    conn = create_engine(string + "?driver='SQL+Server'").connect()
except:
    try:
        conn = create_engine(string + "?driver='ODBC+Driver+18+for+SQL+Server'").connect()
    except:
        import pymssql
        string = f"mssql+pymssql://{username}:{password}@{server}/{database}"   
        conn = create_engine(string).connect() 

In [ ]:
sep_weekly = pd.read_sql(
    """ 
    select date, ticker, closeadj, closeunadj, volume, lastupdated from sep_weekly 
    where date >= '2022-01-01'
    order by ticker, date, lastupdated    
    """,
    conn,
)
sep_weekly = sep_weekly.groupby(["ticker", "date"]).last()
sep_weekly = sep_weekly.drop(columns=["lastupdated"])

ret = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change()
ret.name = "ret"

price = sep_weekly.closeunadj
price.name = "price"

volume = sep_weekly.volume 
volume.name = "volume"

In [ ]:
ret_annual = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change(52)
ret_monthly = sep_weekly.groupby("ticker", group_keys=False).closeadj.pct_change(4)
mom = (1 + ret_annual) / (1 + ret_monthly) - 1
mom.name = "mom"

In [ ]:
weekly = pd.read_sql(
    """ 
    select date, ticker, pb, marketcap, lastupdated from weekly 
    where date>='2022-01-01'
    order by ticker, date, lastupdated    
    """,
    conn,
)
weekly = weekly.groupby(["ticker", "date"]).last()
weekly = weekly.drop(columns=["lastupdated"])

pb = weekly.pb
pb.name = "pb" 
marketcap = weekly.marketcap 
marketcap.name = "marketcap"

In [ ]:
sf1 = pd.read_sql(
    """ 
    select datekey as date, ticker, assets, netinc, equity, lastupdated from sf1
    where datekey>='2022-01-01' and dimension='ARY' and assets>0 and equity>0
    order by ticker, datekey, lastupdated    
    """,
    conn,
)
sf1 = sf1.groupby(["ticker", "date"]).last()
sf1 = sf1.drop(columns=["lastupdated"])

# change dates to Fridays
from datetime import timedelta 
sf1 = sf1.reset_index()
sf1.date =sf1.date.map(
    lambda x: x + timedelta(4 - x.weekday())
)
sf1 = sf1.set_index(["ticker", "date"])
sf1 = sf1[~sf1.index.duplicated()]

assets = sf1.assets
assets.name = "assets" 
netinc = sf1.netinc 
netinc.name = "netinc" 
equity = sf1.equity
equity.name = "equity"

equity = equity.groupby("ticker", group_keys=False).shift() 
roe = netinc / equity 
roe.name = "roe"

assetgr = assets.groupby("ticker", group_keys=False).pct_change()
assetgr.name = "assetgr"

### Merge

- Don't need returns (won't know future return today).

In [ ]:
df = pd.concat(
    (
        mom, 
        volume,
        price, 
        pb, 
        marketcap, 
        roe, 
        assetgr
        ), 
        axis=1
    )
df["roe"] = df.groupby("ticker", group_keys=False).roe.ffill()
df["assetgr"] = df.groupby("ticker", group_keys=False).assetgr.ffill()

df = df.reset_index()
df.date = df.date.astype(str)
df = df[df.date==df.date.max()]
df = df[df.price >= 5]
df = df.dropna()

features = [
    "mom",
    "volume",
    "pb",
    "marketcap",
    "roe",
    "assetgr" 
]

### Industries

In [ ]:
industries = pd.read_sql(
    """ 
    select ticker, famaindustry as industry from tickers   
    """,
    conn,
)
industries["industry"] = industries.industry.fillna("Almost Nothing")
df = df.merge(industries, on="ticker", how="left")

In [ ]:
for x in features:
    df[f"{x}_industry"] = df.groupby(
        ["industry"], 
        group_keys=False
    )[x].apply(
        lambda x: x - x.median()
    )

features += [f"{x}_industry" for x in features]

### Standardize

In [ ]:
for f in features:
    df[f] = df[f].rank(pct=True)

## Load Model, Predict, and Sort

- Sort dataframe so best stocks are at beginning.

In [ ]:
from joblib import load
model = load("mymodel.joblib")
df["predict"] = model.predict(df[features])
df = df.sort_values(by="predict", ascending=False)

## Get data from Alpaca

In [7]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

with open("keys.txt", "r") as f:
    keys = f.readlines()

key = keys[0].strip() 
secret_key = keys[1].strip()
trading_client = TradingClient(key, secret_key, paper=True)

FileNotFoundError: [Errno 2] No such file or directory: 'keys.txt'

### Account equity

In [7]:
account = trading_client.get_account()
equity = float(account.equity)

In [ ]:
with open("keys.txt", "r") as f:
    keys = f.readlines()

key = keys[0].strip() 
secret_key = keys[1].strip()
trading_client = TradingClient(key, secret_key, paper=True)

### Positions

In [ ]:
positions = trading_client.get_all_positions()
if len(positions) > 0:
    df["current"] = {x.symbol: int(x.qty) for x in positions}
    df["current"] = df.current.fillna(0)
else:
    df["current"] = 0

#### Tradable and shortable 

In [8]:
assets = trading_client.get_all_assets()

assets = [
    x for x in assets 
    if (x.asset_class[:]=='us_equity') 
    and (x.symbol in df.index) 
    and (x.status[:]=='active')
]
df["tradable"]= {x.symbol: x.tradable for x in assets}
df["shortable"] = {x.symbol: x.shortable for x in assets}

## Trade

### Rebalance SPY

In [ ]:
import yfinance as yf 
price = yf.download("SPY", start=2024)["Close"].iloc[-1]
shares = int(equity / price)

market_order_data = MarketOrderRequest(
    symbol=tick,
    qty=-df.loc[tick, "trade"],
                side=OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )

#### Calculate target positions

In [11]:
numstocks = 50 ## number long and number short 
dollars_per_stock = 0.4*equity / numstocks


df = df.sort_values(by="rnk")

try:
    short_cutoff = df[df.shortable & (df.bid>0)].rnk.iloc[numstocks-1]
    long_cutoff = df[df.tradable & (df.ask>0)].rnk.iloc[-numstocks]
    df["target"] = np.where(
        df.shortable & (df.bid>0) & (df.rnk<=short_cutoff),
        -short_per_stock/df.bid, 
        0
    )
    df["target"] = np.where(
        df.tradable & (df.ask>0) & (df.rnk>=long_cutoff), 
        long_per_stock/df.ask, 
        df.target
    )
    df["target"] = df.target.astype(int)
except:
    df["target"] = 0

#### Calculate trades

Using a simple but suboptimal protocol: trade to target positions without trying to minimize the number of round trips we might eventually make.

In [12]:
df["trade"] = df.target - df.current

#### Make trades

In [13]:
for tick in df.index: 
    if df.loc[tick, "trade"]<0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=-df.loc[tick, "trade"],
                side=OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
        except:
            print(f"sell order for {tick} failed")
    elif df.loc[tick, "trade"]>0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=df.loc[tick, "trade"],
                side=OrderSide.BUY,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
        except:
            print(f"buy order for {tick} failed")



## Save data

In [14]:
today = datetime.today().strftime("%Y-%m-%d")

df["date"] = today

try:
    d = pd.read_csv("files/trade_data.csv", index_col="ticker")
    d = d[d.date != today]
    df = pd.concat((d, df))
    df.to_csv("files/trade_data.csv")
except:
    df.to_csv("files/trade_data.csv")

In [15]:
account = trading_client.get_account()
account = dict(account)
account = pd.DataFrame(pd.Series(account)).T
account["date"] = today

try:
    d = pd.read_csv("files/account.csv")
    d = d[d.date != today]
    account = pd.concat((d, account))
    account.to_csv("files/account.csv")
except:
    account.to_csv("files/account.csv")


C:\Users\keb7\AppData\Local\Temp\ipykernel_107440\1684087778.py:9: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  account = pd.concat((d, account))


In [16]:
positions = trading_client.get_all_positions()
positions = {x.symbol: x.qty for x in positions}
positions = pd.DataFrame(pd.Series(positions))
positions["date"] = today

try:
    d = pd.read_csv("files/positions.csv")
    d = d[d.date != today]
    positions = pd.concat((d, positions))
    positions.to_csv("files/positions.csv")
except:
    positions.to_csv("files/positions.csv")